Whenever a consumer would like to signal something about IDFM equipments, we propose this person to select an option among common issues (for instance: *"The elevator is not working"*). But we also propose him to specify the matter if needed (for instance : *"There is no light in the elevator"*).

---
---

We thought it would be convenient to categorize a text thanks to a keyword (but unfortunately it is not as good as we thought).

In [2]:
# installation of a keyword extraction library
!pip install -U yake

In [21]:
# load the extraction model to infer keywords according to statistical significance and contextual relevance
import yake

kw_extractor = yake.KeywordExtractor()

# two examples of signaling
text = """J'appuie sur le boutton d'appel, mais ça ne fonctionne pas. Je crois que l'ascenseur est cassé."""
text_2 = """L'escalator était trop rapide quand je l'ai utilisé."""

# defined parameters of the extraction model 
language = "fr"
max_ngram_size = 3
deduplication_threshold = 0.9
numOfKeywords = 1

custom_kw_extractor = yake.KeywordExtractor(lan=language, n = max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)

# most probable keyword
keywords[0][0]

"boutton d'appel"

---
---

Instead, we are going to categorize people signaling with a pre-trained LLM (because we can't classify them without a training data).

In [10]:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python
import os

# import key authentification
API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION')
AZURE_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT') 
API_KEY = os.getenv('AZURE_OPENAI_API_KEY') 

MODEL_NAME = os.getenv('AZURE_OPENAI_MODEL_NAMES')

azure_open_ai_parameters = {
    "api_version": API_VERSION,
    "azure_endpoint": AZURE_ENDPOINT,
    "api_key": API_KEY
}

# use gpt4 deployment to classify signaling
from openai import AzureOpenAI

llm = AzureChatOpenAI(
    **azure_open_ai_parameters,
    temperature=0.9,
)

# define a specific prompt to better classify the signaling
completion = llm.beta.chat.completions.parse(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "You will receive the signaling of an equipment in a train station. Your goal is to categorize that signaling between : Quai, Escalator-fonctionnement, Escalator-bruit, Escalator-propreté, Ascenseur-fonctionnement, Ascenseur-bruit, Ascenseur-propreté"},
        {"role": "user", "content": text},
    ],
    language="fr"
)

signaling_category = completion.choices[0].message.parsed

print(signaling_category)

ModuleNotFoundError: No module named 'openai'

---
---

One of our features propose to retrieve vocal messages. Thus, we use speech recognition to record the consumer trouble.  
We will use OpenAI open source model to meet this need.

In [1]:
# installation of whisper
# https://github.com/openai/whisper

!pip install -U openai-whisper
# installation of a dependency
# !sudo apt update && sudo apt install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 31.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.6/209.6 MB 168.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 163.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 125.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 92.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 152.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 138.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 122.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
import whisper

# load a small model
model = whisper.load_model("base")
# transcription from speech to text
result = model.transcribe("audio.wav", language="fr")

print(result["text"])

/opt/conda/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/opt/conda/li

 La bla bla bla bla bla bla
